The input data for Beagle is a VCF and a panel/linkage map. Beagle would probably be best for high confidence microarray data. GLIMPSE2 is best for seq data and is discussed in the next notebook.

Phasing the filtered, repeat-free panel using BEAGLE and the linkage map:

In [4]:
import numpy as np
import pandas as pd
import subprocess
import re

Generating a beagle reference panel (including multiallelic sites!):

In [11]:
%%bash
source ~/.bashrc
conda activate beagle
bcftools view --samples-file ../evalsamples_IDs.txt \
    ../calls/apal_imputation_panel.vcf.gz \
    -O z > ../wgs_eval.vcf.gz

In [12]:
for chrs in ['Apal_hic_scaffold_10',
 'Apal_hic_scaffold_35',
 'Apal_hic_scaffold_2',
 'Apal_hic_scaffold_30',
 'Apal_hic_scaffold_20',
 'Apal_hic_scaffold_17',
 'Apal_hic_scaffold_4',
 'Apal_hic_scaffold_31',
 'Apal_hic_scaffold_5',
 'Apal_hic_scaffold_15',
 'Apal_hic_scaffold_6',
 'Apal_hic_scaffold_1',
 'Apal_hic_scaffold_21',
 'Apal_hic_scaffold_11']:
    task = "beagle4_pl_impute"
    mem = "128"
    cpus = "20"
    inpath = "../"
    prefix = "wgs_eval"
    outpath = "../calls/"
    refpanel = "../calls/eval_panel.vcf.gz"
    ne = 10000
    chrs = chrs
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         inpath + " " + prefix + " " + outpath + " " + refpanel + " " + str(ne) + " " + chrs], shell=True)

Submitted batch job 8180957
Submitted batch job 8180958
Submitted batch job 8180959
Submitted batch job 8180960
Submitted batch job 8180961
Submitted batch job 8180962
Submitted batch job 8180963
Submitted batch job 8180964
Submitted batch job 8180965
Submitted batch job 8180966
Submitted batch job 8180967
Submitted batch job 8180968
Submitted batch job 8180969
Submitted batch job 8180970


In [48]:
refheader = ['A22241', "Apal-005_kenkel", "A22126", "A22102", "SRR7236001", "M145", "M1_kenkel"]
header = ['A22241_imp', 'Apal-005_kenkel_imp', 'A22126_imp', 
                                          'A22102_imp',  'SRR7236001_imp', 'M145_imp', 'M1_kenkel_imp']
scoring = {"1|1": 0, "0|1": 1, "1|0": 1, "0|0": 2}

ref = pd.read_csv("../calls/eval_groundtruth.vcf.gz", 
                      sep = "\t", header = None, comment = "#", compression = "gzip")
ref.columns = [0, 1, 2, 3, 4, 5, 6, 7, 8] + refheader
ref = ref[[0,1,3,4,7] + refheader]
for samp in refheader:
    ref[samp] = ref[samp].replace(scoring)

imputed = pd.DataFrame()
for chrs in ['Apal_hic_scaffold_10', 'Apal_hic_scaffold_35', 'Apal_hic_scaffold_2', 'Apal_hic_scaffold_30', 'Apal_hic_scaffold_20', 'Apal_hic_scaffold_17', 'Apal_hic_scaffold_4', 'Apal_hic_scaffold_31', 'Apal_hic_scaffold_5', 'Apal_hic_scaffold_15', 'Apal_hic_scaffold_6', 'Apal_hic_scaffold_1', 'Apal_hic_scaffold_21', 'Apal_hic_scaffold_11']:
    vcf = pd.read_csv("../calls/beagle4_wgs_eval_"+ chrs + "_imputed.vcf.gz", 
                      sep = "\t", header = None, comment = "#", compression = "gzip")
    vcf.columns =  [0,1,2,3,4,5,6,7,8] + ['A22241_imp', 'Apal-005_kenkel_imp', 'A22126_imp', 
                                          'A22102_imp',  'SRR7236001_imp', 'M145_imp', 'M1_kenkel_imp']
    vcf["DR2"] = vcf[7].str.split(";", expand = True)[0].str.replace("AR2=", "").astype(float)
    confident = vcf[vcf["DR2"] > 0.3].reset_index(drop=True)
    for samp in header:
        confident[samp] = confident[samp].str.split(":", expand = True)[0].replace(scoring)
    imputed = pd.concat([imputed, confident]).reset_index(drop=True)

imputed.columns = [0,1,2,3,4,5,6,"IMPUTEFIELD",8] + ['A22241_imp', 'Apal-005_kenkel_imp', 'A22126_imp', 
                                          'A22102_imp',  'SRR7236001_imp', 'M145_imp', 'M1_kenkel_imp', "DR2"]

imputed = imputed.merge(ref, how = "left", on = [0,1,3,4]).reset_index(drop=True)

In [56]:
vcf

,0,1,2,3,4,5,6,7,8,A22241_imp,Apal-005_kenkel_imp,A22126_imp,A22102_imp,SRR7236001_imp,M145_imp,M1_kenkel_imp,DR2
0,Apal_hic_scaffold_11,5,.,C,CTAA,.,PASS,AR2=0.23;DR2=0.24;AF=0.26;IMP,GT:DS,0|0:0.36,0|1:0.88,0|0:0.47,0|0:0.34,0|0:0.26,1|0:1,0|0:0.27,0.23
1,Apal_hic_scaffold_11,434,.,A,C,.,PASS,AR2=1.00;DR2=1.00;AF=0.071,GT:DS,0|0:0,0|0:0,0|0:0,1|0:1,0|0:0,0|0:0,0|0:0,1.00
2,Apal_hic_scaffold_11,441,.,A,G,.,PASS,AR2=0.00;DR2=0.00;AF=0,GT:DS,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0.00
3,Apal_hic_scaffold_11,446,.,C,G,.,PASS,AR2=1.00;DR2=1.00;AF=0.071,GT:DS,0|0:0,0|0:0,0|0:0,1|0:1,0|0:0,0|0:0,0|0:0,1.00
4,Apal_hic_scaffold_11,1043,.,ACTCG,A,.,PASS,AR2=0.00;DR2=0.09;AF=0.932;IMP,GT:DS,1|1:1.92,1|1:1.99,1|1:1.67,1|1:1.79,1|1:1.86,1|1:1.98,1|1:1.85,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73593,Apal_hic_scaffold_11,13632933,.,C,T,.,PASS,AR2=0.00;DR2=0.00;AF=0,GT:DS,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0.00
73594,Apal_hic_scaffold_11,13632936,.,G,T,.,PASS,AR2=0.00;DR2=0.00;AF=0,GT:DS,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0.00
73595,Apal_hic_scaffold_11,13633110,.,G,A,.,PASS,AR2=0.00;DR2=0.00;AF=0,GT:DS,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0.00
73596,Apal_hic_scaffold_11,13633114,.,T,C,.,PASS,AR2=0.00;DR2=0.00;AF=0,GT:DS,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0.00


In [49]:
import scipy
def rsquared(x, y):
    """ Return R^2 where x and y are array-like."""

    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return r_value**2

In [50]:
imputed["MAF"] = imputed[7].str.split(";", expand = True)[4].str.replace("MAF=", "").astype(float)
imputed = imputed[imputed["IMPUTEFIELD"].str.contains("IMP")].reset_index(drop=True)

In [51]:
for i in refheader:
    try:
        print(i, rsquared(imputed[i + "_imp"][imputed["DR2"] > 0.9], 
                          imputed[i][imputed["DR2"] > 0.9]), 
              len(imputed[i][imputed["DR2"] > 0.9]))
    except:
        pass

A22241 0.8927424374239843 30651
Apal-005_kenkel 0.8945050082669926 30651
A22126 0.894958495203975 30651
A22102 0.8956700982113909 30651
SRR7236001 0.8779787957322405 30651
M145 0.908007903226031 30651
M1_kenkel 0.9024432666036455 30651


In [52]:
for i in refheader:
    try:
        print(i, rsquared(imputed[i + "_imp"][imputed["DR2"] > 0.9][imputed["MAF"] > 0.25][imputed["MAF"] < 0.51], 
                          imputed[i][imputed["DR2"] > 0.9][imputed["MAF"] > 0.25][imputed["MAF"] < 0.51]), 
              len(imputed[i][imputed["DR2"] > 0.9][imputed["MAF"] > 0.25][imputed["MAF"] < 0.51]))
    except:
        pass

A22241 0.887004522837374 10283
Apal-005_kenkel 0.8990422608466336 10283
A22126 0.8990504217420301 10283
A22102 0.883564063626879 10283
SRR7236001 0.8579605554236056 10283
M145 0.9200479152511241 10283
M1_kenkel 0.9141716531740001 10283


In [53]:
for i in refheader:
    try:
        print(i, rsquared(imputed[i + "_imp"][imputed["DR2"] > 0.4][imputed["MAF"] > 0.25], 
                          imputed[i][imputed["DR2"] > 0.4][imputed["MAF"] > 0.25]), 
              len(imputed[i][imputed["DR2"] > 0.4][imputed["MAF"] > 0.25]))
    except:
        pass

A22241 0.6275634538572525 21375
Apal-005_kenkel 0.6572226336830895 21375
A22126 0.6644744454205584 21375
A22102 0.6414583319837524 21375
SRR7236001 0.6197394200331462 21375
M145 0.6600523008312696 21375
M1_kenkel 0.6653755982490218 21375


In [37]:
!zcat ../calls/beagle4_wgs_eval_Apal_hic_scaffold_10_imputed.vcf.gz | grep "^#" | tail -1

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	A22241	Apal-005_kenkel	A22126	A22102	SRR7236001	M145	M1_kenkel


In [38]:
vcf

,0,1,2,3,4,5,6,7,8,Apal-005_kenkel_imp,A22126_imp,A22102_imp,M1_kenkel_imp,SRR7236001_imp,A22161_imp,M145_imp,DR2
0,Apal_hic_scaffold_10,135,.,A,G,.,PASS,AR2=1.00;DR2=1.00;AF=0.29,GT:DS,0|0:0,0|0:0,0|1:1,1|1:2,0|0:0,1|0:1,0|0:0,1.0
1,Apal_hic_scaffold_10,436,.,A,T,.,PASS,AR2=0.00;DR2=0.00;AF=0,GT:DS,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0.0
2,Apal_hic_scaffold_10,486,.,G,A,.,PASS,AR2=1.00;DR2=1.00;AF=0.21,GT:DS,0|0:0,0|0:0,0|1:1,0|1:1,0|0:0,1|0:1,0|0:0,1.0
3,Apal_hic_scaffold_10,778,.,A,G,.,PASS,AR2=1.00;DR2=1.00;AF=0.14,GT:DS,0|0:0,1|0:1,0|0:0,1|0:1,0|0:0,0|0:0,0|0:0,1.0
4,Apal_hic_scaffold_10,917,.,T,G,.,PASS,AR2=0.00;DR2=0.00;AF=0,GT:DS,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113456,Apal_hic_scaffold_10,25921785,.,A,C,.,PASS,AR2=0.00;DR2=0.00;AF=0,GT:DS,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0|0:0,0.0
113457,Apal_hic_scaffold_10,25922317,.,AG,A,.,PASS,AR2=1.00;DR2=1.00;AF=0.14,GT:DS,0|0:0,0|0:0,0|0:0,1|0:1,0|1:1,0|0:0,0|0:0,1.0
113458,Apal_hic_scaffold_10,25922326,.,T,A,.,PASS,AR2=1.00;DR2=1.00;AF=0.14,GT:DS,0|0:0,0|0:0,1|0:1,0|0:0,0|0:0,0|0:0,0|1:1,1.0
113459,Apal_hic_scaffold_10,25922353,.,G,C,.,PASS,AR2=1.00;DR2=1.00;AF=0.14,GT:DS,0|0:0,0|0:0,1|0:1,0|0:0,0|0:0,0|0:0,0|1:1,1.0
